## DataSet

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
from torchvision import transforms
from glob import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import itertools
import numpy as np
from PIL import Image
T = torch.Tensor
rand_m = np.random.random

In [3]:
from data import Tiny_imagenet_dataset

In [4]:
tt = Tiny_imagenet_dataset(test_mode=True)

In [16]:
tt.image_paths[0].split('_')[0].split('/')[-1]

'n07873807'

In [17]:
a = tt.ohe.transform(np.array(tt.image_paths[0].split('_')[0].split('/')[-1]).reshape(1, -1))

In [18]:
tt.ohe.inverse_transform(a)

array([['n07873807']], dtype='<U9')

In [4]:
train_loader = DataLoader(tt, batch_size=32, shuffle=True)

In [5]:
for x, y in train_loader:
    break

In [6]:
x[0], y

(tensor([[[ 1.0502,  1.0331,  1.0502,  ...,  0.9132,  0.8961,  0.8961],
          [ 1.0502,  1.0331,  1.0502,  ...,  0.9132,  0.8961,  0.8961],
          [ 1.0673,  1.0673,  1.0502,  ...,  0.9132,  0.8961,  0.9303],
          ...,
          [ 0.5878,  0.5878,  0.5707,  ...,  0.3481,  0.3652,  0.3481],
          [ 0.5536,  0.5364,  0.5193,  ...,  0.3823,  0.3823,  0.3652],
          [ 0.5707,  0.5707,  0.5536,  ...,  0.3481,  0.3481,  0.3138]],
 
         [[ 1.5007,  1.4482,  1.4132,  ...,  1.3081,  1.2906,  1.2906],
          [ 1.5007,  1.4657,  1.4132,  ...,  1.2906,  1.2906,  1.2906],
          [ 1.4832,  1.4482,  1.4132,  ...,  1.2906,  1.2906,  1.2731],
          ...,
          [ 0.6779,  0.6779,  0.6604,  ...,  0.4853,  0.5028,  0.5028],
          [ 0.6779,  0.6604,  0.6429,  ...,  0.5203,  0.5203,  0.5203],
          [ 0.7304,  0.6954,  0.6779,  ...,  0.4853,  0.4853,  0.4678]],
 
         [[ 2.0648,  1.9951,  1.9080,  ...,  1.5768,  1.5594,  1.5594],
          [ 2.0300,  1.9603,

## define model and training loop


In [7]:
from torchvision import models

In [8]:
class r_net(nn.Module):
    def __init__(self):
        super(r_net, self).__init__()
        
        self.resnet = models.resnet34(pretrained=True)
        
        self.avgp = nn.AdaptiveAvgPool2d(1)
        self.final_conv = nn.Conv2d(512, 200, 1)
        
    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)
        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)
        x = self.avgp(x)
        x = self.final_conv(x)
        x = x.view(x.shape[0], -1)
        return x

r = r_net().cuda()
r.training

True

In [9]:
r(torch.Tensor(np.random.random((32, 3, 64, 64))).cuda()).shape

torch.Size([32, 200])

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(r.parameters(), lr=0.1)

In [11]:
for epoch in range(20):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        if i % 5 == 0: 
            print("-- ITERATION:", i)
        input, target = data

        input_var = input.cuda()
        target_var = target.cuda()
        
        optimizer.zero_grad()
        
        output = r(input_var)
        
        pred = output
        pred = torch.max(pred, 1)[1]
        
        
        loss = criterion(output, target_var)
        
        # computer gradient + sgd step
        loss.backward()
        
        #for param in r.parameters():
        #    print(param.grad.data.sum())
        
        optimizer.step()

        # print progress
        running_loss += loss.item()
            
            
        if i % 5 == 0:
            print(running_loss / 50)
            running_loss = 0.
            pred = output
            pred = torch.max(pred, 1)[1]
            acc = (pred == target_var).float().mean()
            print(acc.item())

-- ITERATION: 0
0.11201981544494628
0.0
-- ITERATION: 5
3.004381675720215
0.0
-- ITERATION: 0
0.2782748794555664
0.0
-- ITERATION: 5
0.7597996997833252
0.0
-- ITERATION: 0
0.11827223777770995
0.0
-- ITERATION: 5
0.5775546073913574
0.0
-- ITERATION: 0
0.10474189758300781
0.0
-- ITERATION: 5
0.5350306892395019
0.0
-- ITERATION: 0
0.10830462455749512
0.0
-- ITERATION: 5
0.5368499183654785
0.0
-- ITERATION: 0
0.10793811798095704
0.03125
-- ITERATION: 5
0.5434126281738281
0.0
-- ITERATION: 0
0.10196602821350098
0.03125
-- ITERATION: 5
0.537799425125122
0.0
-- ITERATION: 0
0.10878575325012207
0.0
-- ITERATION: 5
0.5317205429077149
0.0
-- ITERATION: 0
0.10826654434204101
0.03125
-- ITERATION: 5
0.5365390777587891
0.0
-- ITERATION: 0
0.10720134735107421
0.0
-- ITERATION: 5
0.5202382659912109
0.03125
-- ITERATION: 0
0.10623207092285156
0.0
-- ITERATION: 5
0.5271868801116943
0.0
-- ITERATION: 0
0.10091756820678711
0.0
-- ITERATION: 5
0.5146277523040772
0.0
-- ITERATION: 0
0.0989979362487793
0.0
